In [1]:
import numpy as np
import os
from PIL import Image
import csv

import re
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import time
import matplotlib.pyplot as plt
import torch.utils.data as data
import json
from torchvision.datasets import ImageFolder
from pathlib import Path

##### Efficient Net V1
from efficientnet_pytorch import EfficientNet

try:
    from tqdm.notebook import tqdm
except ImportError:
    print('tqdm could not be imported. If you want to use progress bar during training,'
          'install tqdm from https://github.com/tqdm/tqdm.')

# File Path

In [2]:
train_img_path = './train_images'
test_img_path = './test_images'
train_label_file = 'train.csv'
test_label_file = 'test.csv'

In [3]:
train_img = []
train_label = []

# Custom Dataset

In [4]:
class AOIDataset(Dataset):
    def __init__(self, img_path, label_filename = None, transform = None):
        self.img_list = []
        self.label = []
        self.transform = transform

        # read label
        if label_filename:
            with open(label_filename, newline = '') as csvfile:
                rows = csv.reader(csvfile)
                for i, row in enumerate(rows):
                    if i: self.label.append(int(row[1]))
            self.label = np.array(self.label)
        
        # img list
        for file in os.listdir(img_path):
            self.img_list.append(img_path + '/' + file)

    def __getitem__(self, index):
        # Read img
        img = Image.open(self.img_list[index])
        if self.transform:
            img = self.transform(img)
        if len(self.label):
            return img, self.label[index]
        return img

    def __len__(self):
        return len(self.img_list)

# dataloader

In [5]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [6]:
dataset = AOIDataset(img_path = train_img_path, label_filename = train_label_file,
                    transform = transform)


In [7]:
train_loader = DataLoader(dataset = dataset, batch_size=48, shuffle=True, 
                        num_workers=0,pin_memory=True)

# Training

In [8]:
device = torch.device('cuda')
torch.backends.cudnn.benchmark = True

In [9]:
# Efficient Net V1 B0
model = EfficientNet.from_pretrained("efficientnet-b0", in_channels = 1, num_classes=6)

model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [10]:
loss = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr = 1e-4)



In [11]:
result_param = {'training_loss': [], 'training_accuracy': []}

for epoch in range(50):

    since = time.time()
    running_training_loss = 0
    running_training_correct = 0
    running_valid_loss = 0
    running_valid_correct = 0
    model.train()
    train_bar = tqdm(train_loader)
    for imgs, label in train_bar:
        imgs = imgs.to(device)
        label = label.to(device, dtype=torch.long)
        optimizer.zero_grad()
        out = model(imgs)
        loss_val = loss(out, label)
        _, pred_class = torch.max(out.data, 1)
        running_training_correct += torch.sum(pred_class == label)
        running_training_loss += loss_val
        loss_val.backward()
        optimizer.step()
        train_bar.set_description(desc='[%d/%d] | Train Loss:%.4f' %
                                        (epoch + 1, 50, loss_val.item()))

    result_param['training_loss'].append(
        running_training_loss.item() / len(dataset))
    result_param['training_accuracy'].append(running_training_correct.item() /
                                                len(dataset))

    print(
        "Epoch:{} Train Loss:{:.4f},  Train Accuracy:{:.4f}".format(
            epoch + 1, result_param['training_loss'][-1], result_param['training_accuracy'][-1])
            )

    now_time = time.time() - since
    print("Training time is:{:.0f}m {:.0f}s".format(
        now_time // 60, now_time % 60))

    # torch.save(model.state_dict(), str(
    #     './checkpoints/' + METHOD + '/' + "EPOCH_" + str(epoch) + ".pkl"))
    # out_file = open(str(
    #     './checkpoints/' + METHOD + '/' + 'result_param.json'), "w+")
    # json.dump(result_param, out_file, indent=4)


  0%|          | 0/53 [00:00<?, ?it/s]

Epoch:1 Train Loss:0.0237,  Train Accuracy:0.6907
Training time is:0m 13s


  0%|          | 0/53 [00:00<?, ?it/s]

Epoch:2 Train Loss:0.0095,  Train Accuracy:0.8754
Training time is:0m 10s


  0%|          | 0/53 [00:00<?, ?it/s]

...

Epoch:48 Train Loss:0.0002,  Train Accuracy:0.9968
Training time is:0m 10s


  0%|          | 0/53 [00:00<?, ?it/s]

Epoch:49 Train Loss:0.0003,  Train Accuracy:0.9953
Training time is:0m 10s


  0%|          | 0/53 [00:00<?, ?it/s]

Epoch:50 Train Loss:0.0002,  Train Accuracy:0.9972
Training time is:0m 10s


# Testing

In [12]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_data = AOIDataset(img_path = test_img_path,
                    transform = transform)

In [13]:
import pandas as pd

In [14]:
submission = pd.read_csv(test_label_file)

In [15]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [16]:
with torch.no_grad():
    model.eval()
    for i, imgs in enumerate(test_data):
        imgs = imgs.to(device)
        out = model(imgs)
        _, pred_class = torch.max(out.data, 1)
        submission.loc[i, 'Label'] = str(pred_class)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 1, 3, 3], but got 3-dimensional input of size [1, 129, 129] instead

In [ ]:
submission.describe()

In [ ]:
submission.to_csv('submissionCNN.csv', index=False)